# Data Wrangling4
jdjhksjhdhd

In [ ]:
Area of Interest

## CopDem
Data manipulation to download, reproject/resampling (3031, 32 ) and mosaic for further use in the analysis. Note the Dem will bve further reprojected to match each DEM we are comparing againts

In [ ]:
## code to create a continenta;l raster for CopDEM
## Produce 3x tiffs for CopDEM reprojected to 3031 with resolutions 32, 10 and 2m for one area of interest.

import os
from osgeo import gdal

# Define the paths to your input folder and output folder
input_folder = '/g/data/yp75/Projects/DEAnt_DEM_Eval/DEMs/forAlex/CopDEM30_AOI_01'
output_folder = '/g/data/yp75/Projects/DEAnt_DEM_Eval/DEMs/forAlex/CopDEM30_AOI_01/Output'

# Define the target projection (EPSG:3031)
target_srs = 'EPSG:3031'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

#List all raster files in the input folder
raster_files = glob.glob(f'{input_folder}/**/*.tif', recursive=True)

##DEM Stitcher Step to output mosaic



for raster_file in raster_files:
    input_raster_path = os.path.join(input_folder, raster_file)
    output_raster_path = os.path.join(output_folder, raster_file)

    # Open the input raster
    input_raster = gdal.Open(input_raster_path)

    # Reproject the raster
    # gdal.Warp(output_raster_path, input_raster, dstSRS=target_srs, dstNodata=-9999, options=['-t_srs', target_srs, '-to', 'VERT_DATUM=' + vertical_datum])

 # Reproject the raster
    gdal.Warp(output_raster_path, input_raster, dstSRS=target_srs, dstNodata=-9999, options=['-t_srs', target_srs, '-to'])

    # Close the input raster
    input_raster = None

    print(f"Reprojected raster saved to {output_raster_path}")

print("All rasters have been reprojected.")




## REMA 32


In [ ]:
#code for Rema 32 

In [ ]:
#code for CopDEM resampling to Reema32 grid